<a href="https://colab.research.google.com/github/JALVARADORUIZ/Magister_BigData_UB/blob/main/M2_AG1_GRUPO14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **FUNDAMENTOS DE ESTADÍSTICA**
PADAWANS:
- PAULA VALLEJO VARGAS
- JOSE ARMANDO ALVARADO RUIZ

# **Introducción al cálculo de probabilidades**



**Entrega: M2_AG1_CORONAVIRUS**

# Descripción de la tarea


Tenéis un dataset (worlddometer_covid_ue.csv) con los datos a nivel europeo del Covid-19. En el momento del estudio, se detecta la siguiente situación:

Hay 6 países europeos con el mayor número de casos por Covid-19 que son Rusia, España, UK, Francia, Italia y Alemania con, aproximadamente, 250.000 casos.

Por lo que sabemos, se han adoptado algunas medidas similares de contención (confinamiento, mascarilla, geles, etc.) aunque no se han aplicado en las mismas fechas (por ejemplo, Italia comenzó 2 semanas antes que España y, en UK, postergaron el confinamiento en más de 15 días con respecto a España).

El número de habitantes por población es similar, quitando a Rusia que tiene el triple de personas que en España. Esto hace que descartemos a Rusia del análisis comparativo. En este punto, cabría la opción de plantearse añadir a Ucrania que tiene un número similar de habitantes que España.

Con los datos facilitados sobre el total de casos confirmados con Covid-19 (suponemos que con PCR) y sobre el total de fallecimientos acontecidos, te pedimos que halles:

- La probabilidad de sobrevivir en cualquiera de los seis países europeos seleccionados para el estudio.
- Si seleccionamos una de las personas fallecidas, ¿cuál es la probabilidad de que residiera en España? ¿Y en el resto de países?

Justifica y comenta los resultados obtenidos.

CONSIDERACIONES

Fuente de la información.

Se supondrá que los datos de fallecimiento se han contabilizado exactamente igual en los países seleccionados para el análisis.

Los datos son oficiales, con fecha 29-08-2020, pero esto no quiere decir que se puedan extraer conclusiones definitivas, mejor dicho, hay que considerar más variables de las que se tienen en cuenta en este análisis como, por ejemplo, la distribución de la edad en la población, la forma de considerar los fallecimientos por Covid-19 (si son con PCR o no todos los que figuran), las medidas adoptadas, etc.

Justifica adecuadamente tus respuestas.

# **Carga y análisis de la Data**

IMPORTACIÓN DE LIBRERÍAS

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
import statsmodels.api as sm
import seaborn as sns
import math
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
pd.options.mode.chained_assignment = None
%matplotlib inline
import pandas.util.testing as tm

# Carga de Datos y visualización general

url = 'https://raw.githubusercontent.com/md-lorente/data/master/worlddometer_covid_ue.csv'
#df = pd.read_csv(url, sep=";", decimal=",")
df = pd.read_csv(url, sep=';',decimal = '.', thousands= ',')
df = df.infer_objects()
df.head(10)

,Num,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,Serious Critical,Total Cases per 1M pop,Deaths per 1M pop,Total Tests,Tests per 1M pop,Population
0,0,Europe,3543710,10493.0,207347.0,149.0,2063335.0,1273028.0,5619.0,NaN,NaN,NaN,NaN,NaN
1,1,Russia,990326,498.0,17093.0,68.0,806982.0,166251.0,23.0,6786.0,117.0,36400000.0,249409.0,145944843.0
2,2,Spain,455621,NaN,29011.0,NaN,NaN,NaN,751.0,9744.0,620.0,8517446.0,182161.0,46757783.0
3,3,UK,332752,NaN,41498.0,NaN,NaN,NaN,60.0,4897.0,611.0,16273209.0,239508.0,67944393.0
4,4,France,272530,NaN,30602.0,NaN,86177.0,155751.0,387.0,4174.0,469.0,6175223.0,94571.0,65297356.0
5,5,Italy,266853,NaN,35473.0,NaN,208224.0,23156.0,79.0,4415.0,587.0,8509618.0,140778.0,60447003.0
6,6,Germany,242825,NaN,9363.0,NaN,217484.0,15978.0,245.0,2897.0,112.0,11208091.0,133703.0,83828051.0
7,7,Ukraine,119074,2096.0,2527.0,35.0,56734.0,59813.0,177.0,2725.0,58.0,1536171.0,35161.0,43689434.0
8,8,Romania,86785,952.0,3578.0,39.0,37665.0,45542.0,492.0,4516.0,186.0,1795633.0,93445.0,19216003.0
9,9,Belgium,84599,647.0,9891.0,5.0,18403.0,56305.0,72.0,7294.0,853.0,2261043.0,194953.0,11597901.0


In [44]:
df.columns

Index(['Num', 'Country', 'Total Cases', 'New Cases', 'Total Deaths',
       'New Deaths', 'Total Recovered', 'Active Cases', 'Serious Critical',
       'Total Cases per 1M pop', 'Deaths per 1M pop', 'Total Tests',
       'Tests per 1M pop', 'Population '],
      dtype='object')

In [45]:
df.dtypes

Num                         int64
Country                    object
Total Cases                 int64
New Cases                 float64
Total Deaths              float64
New Deaths                float64
Total Recovered           float64
Active Cases              float64
Serious Critical          float64
Total Cases per 1M pop    float64
Deaths per 1M pop         float64
Total Tests               float64
Tests per 1M pop          float64
Population                float64
dtype: object

# **Seleccion Universo**

El universo está formado por estos 6 países: 

- España
- UK
- Francia
- Italia
- Alemania
- Ucrania.

In [46]:
# Seleccionamos a los 6 paises que serán nuestro Universo
df2 = df.loc[2:7]

In [47]:
df2

,Num,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,Active Cases,Serious Critical,Total Cases per 1M pop,Deaths per 1M pop,Total Tests,Tests per 1M pop,Population
2,2,Spain,455621,NaN,29011.0,NaN,NaN,NaN,751.0,9744.0,620.0,8517446.0,182161.0,46757783.0
3,3,UK,332752,NaN,41498.0,NaN,NaN,NaN,60.0,4897.0,611.0,16273209.0,239508.0,67944393.0
4,4,France,272530,NaN,30602.0,NaN,86177.0,155751.0,387.0,4174.0,469.0,6175223.0,94571.0,65297356.0
5,5,Italy,266853,NaN,35473.0,NaN,208224.0,23156.0,79.0,4415.0,587.0,8509618.0,140778.0,60447003.0
6,6,Germany,242825,NaN,9363.0,NaN,217484.0,15978.0,245.0,2897.0,112.0,11208091.0,133703.0,83828051.0
7,7,Ukraine,119074,2096.0,2527.0,35.0,56734.0,59813.0,177.0,2725.0,58.0,1536171.0,35161.0,43689434.0


# **Probabilidad de Laplace**

**Pregunta 1:**

La probabilidad de sobrevivir en cualquiera de los seis países europeos seleccionados para el estudio.

Considerando los datos disponibles, consideraremos que la Probabilidad de sobrevivir esta dada por:

        1 - Probabilidad de Morir

donde Probabilidad de Morir es *total_deaths/total_cases*, por lo tanto:

        Probabilidad de sobrevivir = 1 - (total_deaths/total_cases)

En este caso, estamos utilizando la Probabilidad a traves de Laplace, ya que bastaría con determinar la proporción entre el número de casos favorables de un suceso y el número de resultados posibles relativos al experimento, donde los casos favorables serían el total de muertes y el número posible es el total de casos, basado en los datos facilitados sobre el total de casos confirmados con Covid-19 (suponemos que con PCR) y sobre el total de fallecimientos acontecidos.

In [48]:
# Realizamos copia del dataset con solo las variables que nos interesan para este analisis

df_universo = df2[['Country','Total Cases','Total Deaths']]

def laplace_pais(country,df):
  prob_morir = df.loc[df['Country']==country]['Total Deaths'] / df.loc[df['Country']==country]['Total Cases']
  prob_sobrevivir = 1 - prob_morir.values[0]
  return round(prob_sobrevivir,3)


print("La Probabilidad de Sobrevivir en España es de : ", laplace_pais('Spain',df_universo))
print("La Probabilidad de Sobrevivir en Reino Unido es de : ", laplace_pais('UK',df_universo))
print("La Probabilidad de Sobrevivir en Francia es de : ", laplace_pais('France',df_universo))
print("La Probabilidad de Sobrevivir en Italia es de : ", laplace_pais('Italy',df_universo))
print("La Probabilidad de Sobrevivir en Alemania es de : ", laplace_pais('Germany',df_universo))
print("La Probabilidad de Sobrevivir en Ucrania es de : ", laplace_pais('Ukraine',df_universo))


La Probabilidad de Sobrevivir en España es de :  0.936
La Probabilidad de Sobrevivir en Reino Unido es de :  0.875
La Probabilidad de Sobrevivir en Francia es de :  0.888
La Probabilidad de Sobrevivir en Italia es de :  0.867
La Probabilidad de Sobrevivir en Alemania es de :  0.961
La Probabilidad de Sobrevivir en Ucrania es de :  0.979


Analisis de Resultados

Podemos ver que la mayor probabilidad de sobrevivir al Coronavirus dentro de nuestro universo de 6 países seleccionados esta en Ucrania con un 97,9%, mientras que la menor probabilidad la tiene es Italia con un 86,7%.

A continuación un cuadro comparativo de nuestra universo:



# **Teorema de la Probabilidad Total y Teorema de Bayes**

Para responder la pregunta 2, utilizaremos el Teorema de Probabilidad Total y Teorema de Bayes.


Pregunta 2:

Si seleccionamos una de las personas fallecidas, ¿cuál es la probabilidad de que residiera en España? ¿Y en el resto de países?

In [49]:
# Total de Casos universo, donde Porcentaje Casos es el total de casos de cada país dividido el total de casos del Universo.

total_casos_universo = df_universo['Total Cases'].sum()
df_universo['Porcentaje Casos'] = df_universo['Total Cases'] / total_casos_universo

# Aprovechamos de crear una nueva columna probabilidad de morir por país para los siguientes analisis.

df_universo['Probabilidad morir'] = df_universo['Total Deaths'] / df_universo['Total Cases']

In [50]:
df_universo

,Country,Total Cases,Total Deaths,Porcentaje Casos,Probabilidad morir
2,Spain,455621,29011.0,0.269653,0.063674
3,UK,332752,41498.0,0.196935,0.124711
4,France,272530,30602.0,0.161293,0.112289
5,Italy,266853,35473.0,0.157933,0.132931
6,Germany,242825,9363.0,0.143713,0.038559
7,Ukraine,119074,2527.0,0.070472,0.021222


Esto refleja que el 26,9% de los casos de nuestro universo estan en España, mientras que en UK tenemos 19,6% de los casos, mientras que en Ucrania solo corresponde al 7% de los casos.

Para calcular la probabilidad de que una persona muera en el universo de seis países P(M), esta se puede calcular por la propiedad de la probabilidad total, esto es:

$$ 
P(M) = \sum_{j=1}^6 P(A_{j}) P(M|A_{j}) =  P(A_{1}) \cdot P(M|A_{1}) + P(A_{2}) \cdot P(M|A_{2}) + \ldots + P(A_{6}) \cdot P(M|A_{6})
\\  $$

Donde A es cada uno de los 6 países.

In [51]:
def probabilidad_total(df):
  prob_total = 0
  for index, row in df.iterrows():
    prob_total = prob_total + row['Porcentaje Casos']*(row['Probabilidad morir'])
  return prob_total

probabilidad_total = probabilidad_total(df_universo)

print("La probabilidad total en nuestro universo es de : ", round(probabilidad_total,3))

La probabilidad total en nuestro universo es de :  0.088


La probabilidad total se refiere principalmente a que existe un 8,8% de probabilidades de fallecer dentro de nuestro universo que corresponde a los seís países.

Ahora podemos utilizar el Teorema de Bayes para determinar la probabilidad de que esa persona muerta resida en un país especifico, utilizando esta formula:

$$ 
P(País|M) = \frac{ P(país) \cdot P(M|país) }{ Probabilidad   total }
\\ \\
$$

In [52]:
def teorema_bayes(country, df, prob_total):
  numerador = df.loc[df['Country']==country]['Porcentaje Casos']*(df.loc[df['Country']==country]['Probabilidad morir'])
  bayes = numerador/prob_total
  return round(bayes.values[0],4)

print("Probabilidad de que persona fallecida resida en España es: ", teorema_bayes('Spain', df_universo, probabilidad_total))
print("Probabilidad de que persona fallecida resida en Reino Unido es: ",teorema_bayes('UK', df_universo, probabilidad_total))
print("Probabilidad de que persona fallecida resida en Francia es: ",teorema_bayes('France', df_universo, probabilidad_total))
print("Probabilidad de que persona fallecida resida en Italia es: ",teorema_bayes('Italy', df_universo, probabilidad_total))
print("Probabilidad de que persona fallecida resida en Alemania es: ",teorema_bayes('Germany', df_universo, probabilidad_total))
print("Probabilidad de que persona fallecida resida en Ucrania es: ",teorema_bayes('Ukraine', df_universo, probabilidad_total))


Probabilidad de que persona fallecida resida en España es:  0.1954
Probabilidad de que persona fallecida resida en Reino Unido es:  0.2795
Probabilidad de que persona fallecida resida en Francia es:  0.2061
Probabilidad de que persona fallecida resida en Italia es:  0.2389
Probabilidad de que persona fallecida resida en Alemania es:  0.0631
Probabilidad de que persona fallecida resida en Ucrania es:  0.017


# Árbol de decisión

Otra forma de obtener el Teorema de Bayes es a través del Árbol de decisión, que es una técnica muy sencilla que, además, se usa para tomas de decisión.

